In [ ]:
import ast
import pandas as pd 
import timeit
import numpy as np 
from statistics import mean
import prophet
import pydeck as pdk

# Imports

In [ ]:
#Import two data-sets
starbucks_df = pd.read_csv('./Clean_Data/London_Starbucks_Data.csv')
airbnb_df = pd.read_csv('./Clean_Data/London_AirBNB_Data.csv')
#Convert List columns stored as strings to lists
starbucks_df['airbnb_distances'] = starbucks_df['airbnb_distances'].apply(ast.literal_eval)
airbnb_df['starbucks_distances'] = airbnb_df['starbucks_distances'].apply(ast.literal_eval)
#Convert Date columns to Datetime
airbnb_df['first_review'] = pd.to_datetime(airbnb_df['first_review'])
airbnb_df['last_review'] = pd.to_datetime(airbnb_df['last_review'])
#Preview to verify
display(starbucks_df.head(1))
display(starbucks_df.info())

display(airbnb_df.head(1))
display(airbnb_df.info())

# Data Analytics

In [ ]:
#Manually assigned distances of interest
bins = [.1, .25, .5, 1, 2, 5]
#Define a function that loops through each 'bin' (not technically correct)
#and counts starbucks within that range, then populate a column with that count
#for future plots
def airbnb_to_starbucks():
    for bin in bins:
        def nested_bin(row):
            dist_to_sb_list = row['starbucks_distances']
            sb_within_range = [x for x in dist_to_sb_list if x < bin]
            return len(sb_within_range)
        column_name = f"SB_within_{bin}_miles"
        airbnb_df[column_name] = airbnb_df.apply(nested_bin, axis=1)
# Run the function
airbnb_to_starbucks()
# Do it again for starbucks
def starbucks_to_airbnb():
    for bin in bins:
        def nested_bin(row):
            dist_to_bnb_list = row['airbnb_distances']
            bnb_within_range = [x for x in dist_to_bnb_list if x < bin]
            return len(bnb_within_range)
        column_name = f"Airbnb_within_{bin}_miles"
        starbucks_df[column_name] = starbucks_df.apply(nested_bin, axis=1)
starbucks_to_airbnb()

In [ ]:
#calculate the average distance to the nearest 5 starbucks or bnbs
def mean_dist_nearest_5_sb(row):
    dist_list = row['starbucks_distances']
    dist_list.sort()
    return round(mean(dist_list[:5]),2)

def mean_dist_nearest_5_bnb(row):
    dist_list = row['airbnb_distances']
    dist_list.sort()
    return round(mean(dist_list[:5]),2)

airbnb_df['nearest_5_avg'] = airbnb_df.apply(mean_dist_nearest_5_sb, axis=1)
starbucks_df['nearest_5_avg'] = starbucks_df.apply(mean_dist_nearest_5_bnb, axis=1)

In [ ]:
#Add a column for "price per occupant" 
airbnb_df['price_per_occupant'] = round(airbnb_df['price']/airbnb_df['accommodates'],2)

In [ ]:
first_review_df = airbnb_df.copy()
first_review_df.dropna(how='any', inplace=True)
first_review_df.set_index('first_review',inplace=True)
first_review_df_agg = first_review_df.groupby([first_review_df.index.year, first_review_df.index.month])
first_review_df_agg = first_review_df_agg['id'].count().to_frame()
first_review_df_agg.columns = ['Total_New_Airbnbs']
first_review_df_agg['dateseries'] = pd.to_datetime(first_review_df_agg.index.get_level_values(0).astype(str)+'-'+first_review_df_agg.index.get_level_values(1).astype(str))
first_review_df_agg.set_index('dateseries',inplace=True)
first_review_df_agg.reset_index(inplace=True)
first_review_df_agg.columns = ['ds','y']
# first_review_df_agg.to_csv('./Clean_Data/DateSeries_Prophet.csv')


In [ ]:
def total_bnbs(row):
    date = row['ds']
    total = first_review_df_agg.loc[first_review_df_agg['ds'] <= date]['y'].sum()
    return total

first_review_df_agg['total_airbnbs'] = first_review_df_agg.apply(total_bnbs, axis=1)

total_bnb_df = first_review_df_agg[['ds','total_airbnbs']].rename(columns={'total_airbnbs':'y'})
new_bnb_df = first_review_df_agg[['ds','y']]

In [ ]:
#model and plot for new airbnbs
m = prophet.Prophet()
m.fit(new_bnb_df)
future_df = m.make_future_dataframe(periods=60, freq='M')
forecast = m.predict(future_df)
m.plot_components(forecast)

In [ ]:
m.plot(forecast)

In [ ]:
#model and plot for total airbnbs
m2 = prophet.Prophet()
m2.fit(total_bnb_df)
future_df2 = m2.make_future_dataframe(periods=60, freq='M')
forecast2 = m2.predict(future_df2)
m2.plot_components(forecast2)

# Visualizations

### Reviews with respect to the number of starbucks within 0.5 miles (distance may change based on group-by line)

In [ ]:
#Aggregate reviews by the number of SB within X miles
airbnb_df_reviewed_only = airbnb_df.loc[airbnb_df['review_scores_rating'] != -1]
aggregate_bnb_df = airbnb_df_reviewed_only.groupby(by=['SB_within_0.5_miles'])
agg_bnb_df_mean = aggregate_bnb_df[['price', 'review_scores_rating','review_scores_location']].mean()
display(aggregate_bnb_df['review_scores_rating'].count())
agg_bnb_df_mean[['review_scores_rating','review_scores_location']].plot(figsize=(12,8), ylim=(4.3,5))

### Not very helpful scatter-plot of reviews vs average distance to nearest 5 SB

In [ ]:
airbnb_df.loc[(airbnb_df['review_scores_rating'] != -1) & (airbnb_df['number_of_reviews'] > 10)]\
    .plot(x='nearest_5_avg', y='review_scores_rating', kind='scatter', ylim=(4.1, 5.1), figsize=(12,8), s=1)

### Chart Depicting Reviews vs mileage to the nearest 5 starbucks

In [ ]:
#actual binning of the data by nearest_5_avg
nearest_5_bins = [0, 0.25, 0.5, 1, 2, 5]
labels = [1,2,3,4,5]
airbnb_df['nearest_5_distance_bins']=pd.cut(airbnb_df['nearest_5_avg'], bins=nearest_5_bins, labels=labels)
grouped_near5_airbnb = airbnb_df.loc[airbnb_df['review_scores_rating'] != -1].groupby('nearest_5_distance_bins')
plt = grouped_near5_airbnb[['review_scores_rating', 'review_scores_location']].mean().plot(figsize=(12,6))
plt.set_xticks([0,1,2,3,4])
plt.set_xlim((-.1,4.1))
plt.set_xticklabels(['<.25','0.25 - 0.5','0.5 - 1.0','1.0 - 2.0','2.0 - 5.0'])
plt.set_xlabel("Distance Bins (miles)")
plt.set_ylabel("Average Review")

### Average Price with respect to distance to nearest 5 starbucks (same bins as above)

In [ ]:
plt2 = grouped_near5_airbnb[['price_per_occupant']].mean().plot(figsize=(12,6))
plt2.set_xticks([0,1,2,3,4])
plt2.set_xticklabels(['<.25','0.25 - 0.5','0.5 - 1.0','1.0 - 2.0','2.0 - 5.0'])
plt2.set_xlabel("Distance Bins (miles)")
plt2.set_ylabel("Average Price ($ Per Accomodated Guest)")

# Testing that supported decisions while writing

In [ ]:
#speed testing list comprehension vs. numpy eval:
def numpy_speed():
    data_list = starbucks_df['airbnb_distances'][0]
    numpy_array = np.array(data_list)
    nearby_distances = numpy_array[numpy_array < 1]
    nearby_distances = np.sort(nearby_distances)
    nearby_count = len(nearby_distances)
    return(nearby_count,nearby_distances)

time_taken_numpy = timeit.timeit(numpy_speed, number=5000)

def list_speed():
    data_list = starbucks_df['airbnb_distances'][0]
    keep_list = [x for x in data_list if x < 1]
    keep_list.sort()
    nearby_count = len(keep_list)
    return(nearby_count,keep_list)

time_taken_list = timeit.timeit(list_speed, number=1000)

print(f"numpy: {time_taken_numpy}")
print(f"list comprehension: {time_taken_list}")

#list comprehension was consistently ~5-10% faster

In [ ]:
#Prepping Data into lat and long bins for geographical plotting
pydeck_df = airbnb_df[['latitude','longitude']].copy()
lat_min = pydeck_df['latitude'].min()
lat_max = pydeck_df['latitude'].max()
long_min = pydeck_df['longitude'].min()
long_max = pydeck_df['longitude'].max()
bin_count = 30

#Generate Bin Limits
lat_step = round((lat_max-lat_min)/bin_count,3)
long_step = round((long_max-long_min)/bin_count,3)
lat_bins = np.arange(lat_min,lat_max,lat_step)
long_bins = np.arange(long_min,long_max,long_step)

#The following solves some rounding errors that didn't bin data
lat_bins[0] -= .01
lat_bins[-1] += .01
long_bins[0] -= .01
long_bins[-1] += .01

#Generate Grid Centers
lat_centers = []
for index, item in enumerate(lat_bins):
    try:
        coord_center = (lat_bins[index] + lat_bins[index+1])/2
        lat_centers.append(coord_center)
    except IndexError:
        pass

long_centers = []
for index, item in enumerate(lat_bins):
    try:
        coord_center = (long_bins[index] + long_bins[index+1])/2
        long_centers.append(coord_center)
    except IndexError:
        pass
#Populate bins
pydeck_df['lat_bins'] = pd.cut(pydeck_df['latitude'], bins=lat_bins, labels=range(len(lat_bins)-1))
pydeck_df['long_bins'] = pd.cut(pydeck_df['longitude'], bins=long_bins, labels=range(len(long_bins)-1))

#Generate Pixels
pixel_pairs = []
for latitude in lat_centers:
    for longitude in long_centers:
        pair = [latitude, longitude]
        pixel_pairs.append(pair)

index_pairs = []
for x in range(len(lat_centers)):
    for y in range(len(long_centers)):
        pair = [x,y]
        index_pairs.append(pair)
pydeck_summary_df = pd.DataFrame({'xy_pair':index_pairs,'latlong_pair':pixel_pairs})
def pixel_counter(row):
    bin = row['xy_pair']
    count = len(pydeck_df.loc[(pydeck_df['lat_bins'] == bin[0]) & (pydeck_df['long_bins'] == bin[1])])
    return count

pydeck_summary_df['AirBnb_Count'] = pydeck_summary_df.apply(pixel_counter, axis=1)
pydeck_summary_df['lat'] = pydeck_summary_df['latlong_pair'].apply(lambda x: x[0] if x else None)
pydeck_summary_df['long'] = pydeck_summary_df['latlong_pair'].apply(lambda x: x[1] if x else None)
pydeck_summary_df = pydeck_summary_df.loc[pydeck_summary_df['AirBnb_Count'] > 0 ]

In [ ]:
pydeck_summary_df

In [ ]:
#speed testing list comprehension vs. numpy eval:
def numpy_speed():
    data_list = starbucks_df['airbnb_distances'][0]
    numpy_array = np.array(data_list)
    nearby_distances = numpy_array[numpy_array < 1]
    nearby_distances = np.sort(nearby_distances)
    nearby_count = len(nearby_distances)
    return(nearby_count,nearby_distances)

time_taken_numpy = timeit.timeit(numpy_speed, number=5000)

def list_speed():
    data_list = starbucks_df['airbnb_distances'][0]
    keep_list = [x for x in data_list if x < 1]
    keep_list.sort()
    nearby_count = len(keep_list)
    return(nearby_count,keep_list)

time_taken_list = timeit.timeit(list_speed, number=1000)

print(f"numpy: {time_taken_numpy}")
print(f"list comprehension: {time_taken_list}")

#list comprehension was consistently ~5-10% faster

In [ ]:
#Learning how to use pydeck
# Build the map layer:    
airbnbs = pdk.Layer(type='ColumnLayer',
                  data=pydeck_summary_df,
                  opacity=0.3 ,
                  get_position=['long','lat'],
                  get_elevation='AirBnb_Count',
                  auto_highlight=True,
                  elevation_scale=2,
                  pickable=True,
                  get_fill_color=['AirBnb_Count', 130],
                  coverage=12/(bin_count))

starbucks = pdk.Layer(
    "HeatmapLayer",
    data=starbucks_df,
    opacity=0.8,
    get_position=["longitude", "latitude"],
    threshold=0.75,
    aggregation=pdk.types.String("SUM"),
    get_weight=4,
    pickable=True,
)

# Set the view parameters:
view_state = pdk.ViewState(longitude=pydeck_summary_df['long'].mean(), 
                           latitude=pydeck_summary_df['lat'].mean(),
                           zoom=10,
                           min_zoom=9,
                           max_zoom=16,
                           pitch=45.0,
                           bearing=30)

# Render the map:
r = pdk.Deck(layers=[airbnbs, starbucks], initial_view_state=view_state, map_style=pdk.map_styles.CARTO_DARK_NO_LABELS)
r.to_html('london_airbnb.html')

In [ ]:
import zipfile
import os

In [ ]:
# File paths
file_to_zip = './london_airbnb.html'
zip_file_name = 'london_map.zip'

# Create a new zip file
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(file_to_zip, arcname='file.txt')

print("File zipped successfully")
#File is ignored, so no need to delete, but an option
# os.remove('london_airbnb.html')


'c:\\Users\\matt.dipinto\\DataClass\\m.c-squared'